In [ ]:
!pip3 install torch torchvision
!pip3 install split-folders tqdm
!pip3 install opencv-python
!pip3 install Pillow

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# copy zip file from drive
!cp /content/drive/MyDrive/DLFHC/CXR.zip ./CXR_copy.zip # data expected to be in DLFHC folder

In [ ]:
import os
import random
import torch
import numpy as np
import torch.nn.functional as F
import math   
import torchvision
import matplotlib.pyplot as plt
import splitfolders
import pandas as pd
import cv2
from sklearn import preprocessing
from torch import nn
from torchvision import datasets, transforms, models
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score
from PIL import Image

# set seed
seed = 21
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
# use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# function for applying grayscale sobel transformation 
def gray_sobel_transform(img, sobel_kernel = 3):
    # from: https://www.programcreek.com/python/?project_name=PacktPublishing%2FOpenCV-3-x-with-Python-By-Example via https://www.programcreek.com/python/example/85270/cv2.CV_64F 
    
    # usually grayscaled, but this cannot be turned back
    grayscaled = 255 * np.array(img).astype('uint8') # img # cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # for defining orientations
    # use cv2.CV_64F for kernel
    # apply both directions
    sobel_x = cv2.Sobel(grayscaled,cv2.CV_32F, 1, 0, ksize= sobel_kernel) 
 
    sobel_y = cv2.Sobel(grayscaled,cv2.CV_32F, 0, 1, ksize= sobel_kernel) 

    # scaled
    abs_sobel_x = cv2.convertScaleAbs(sobel_x) 
    abs_sobel_y = cv2.convertScaleAbs(sobel_y) 

    # Weighted sums to combine x and y
    weighted_image = cv2.addWeighted(abs_sobel_x, 0.5, abs_sobel_y, 0.5, 0)
    

    #ALTERNATIVE EDGE FINDING: LAPLACIAN (basically subtract the gaussian blur)
    # weighted_image = cv2.Laplacian(grayscaled,cv2.CV_64F)


    # and then return the image
    return Image.fromarray(weighted_image)

In [ ]:
# function for unzipping and splitting data
def split_data(isBinary):
  !unzip -q ./CXR_copy.zip

  if isBinary:
    # delete pneumonia folder for binary classes
    print('removing pneumonia images')
    !rm -rf ./CXR/pneumonia_CXR

  # train validate test split
  splitfolders.ratio('./CXR', output='./splitCXR', seed=seed, ratio=(.7, .1, .2), group_prefix=None)

In [ ]:
# function to clean up copied data
def clean_up():
  !rm -rf ./CXR ./splitCXR

In [ ]:
# function to get transforms based on transformation provided
def get_transform(transformation):
  # test/validation data converted to tensor, expected size, and normalized
  test_transform = transforms.Compose([
                                transforms.Resize((224,224)),
                                transforms.ToTensor(), 
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])
  if transformation == 'Sobel':
    # From https://discuss.pytorch.org/t/how-to-use-generic-transforms-for-the-following-preprocessing-step/50658
    train_transform = transforms.Compose([
                                  transforms.Resize((224,224)),
                                  transforms.Lambda(gray_sobel_transform), # Sobel!
                                  transforms.ToTensor(),                          
                                  transforms.Normalize(mean = (0,)*3, std = (0.1)*3)                                
                                ])
  elif transformation == 'Gaussian':
    train_transform = transforms.Compose([
                                  transforms.Resize((224,224)),
                                  transforms.ToTensor(), 
                                  transforms.GaussianBlur(11, sigma=(0.1, 2.0)),
                                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])
  elif transformation == 'Random Perspective & Vertical Flip':
    train_transform = transforms.Compose([transforms.Resize((224,224)),
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomPerspective(),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                               ])
  elif transformation == 'Unaugmented':
    train_transform = test_transform
  else:
    raise Exception('Invalid tranformation')
  return train_transform, test_transform

In [ ]:
# function for returning data loaders based on transformation
def get_data_loaders(transformation):
  train_transform, test_transform = get_transform(transformation)

  train_data = datasets.ImageFolder('./splitCXR/train', transform=train_transform)
  val_data = datasets.ImageFolder('./splitCXR/val', transform=test_transform)
  test_data = datasets.ImageFolder('./splitCXR/test', transform=test_transform)

  # Uncomment to use subset
  # train_size = 200
  # train_data = torch.utils.data.Subset(train_data, np.random.choice(len(train_data), train_size, replace=False))
  # val_data = torch.utils.data.Subset(val_data, np.random.choice(len(val_data), math.floor(train_size * .2), replace=False))
  # test_data = torch.utils.data.Subset(test_data, np.random.choice(len(test_data),  math.floor(train_size * .1), replace=False))

  print('Number of training images: {:.0f}'.format(len(train_data)))
  print('Number of validation images: {:.0f}'.format(len(val_data)))
  print('Number of test images: {:.0f}'.format(len(test_data)))

  training_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=32, shuffle=True)
  validation_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=32, shuffle=False)
  test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=32, shuffle=False)

  return training_loader, validation_loader, test_loader

In [ ]:
# function for displaying image
def imshow(img, title):
    npimg = img.numpy()
    plt.figure(figsize=(15, 7))
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.show()
# function for displaying batch of images
def show_batch_images(dataloader):
    images, labels = next(iter(dataloader))
    img = torchvision.utils.make_grid(images, padding=25)
    imshow(img, title=None)

In [ ]:
# function for training a model and testing on validation data
def train(model, epochs, lr, training_loader, validation_loader):
  # use cross entropy loss and adam optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = lr)

  # intitialing variables to store running values
  running_loss_history = []
  running_correct_history = []

  val_running_loss_history = []
  val_running_corrects_history = []
  for i in range(epochs):
    running_loss = 0.0
    running_correct = 0.0
    val_running_loss = 0.0
    val_running_corrects = 0.0

    # loop to train model
    for inputs, labels in training_loader:
      # pass to device
      inputs = inputs.to(device)
      labels = labels.to(device)

      # calculate predictions
      outputs = model.forward(inputs)

      # calculate loss and update parameters
      loss = criterion(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # get predicted class
      _, preds = torch.max(outputs,1)

      # add values to running totals
      running_correct += torch.sum(preds == labels.data)
      running_loss += loss.item()
    else:
      # disabled to improve performance
      with torch.no_grad():
        # loop to test validation dataset
        for val_inputs, val_labels in validation_loader:
          # pass to device
          val_inputs = val_inputs.to(device)
          val_labels = val_labels.to(device)

          # calculate predictions
          val_outputs = model.forward(val_inputs)
          val_loss = criterion(val_outputs, val_labels)

          # get predicted class for validation 
          _, val_preds = torch.max(val_outputs,1)

          # store running totals on validation data
          val_running_corrects += torch.sum(val_preds == val_labels.data)
          val_running_loss += val_loss.item()

      # compute loss and accuracy for current epoch
      epoch_loss = running_loss / len(training_loader.dataset)
      epoch_accuracy = running_correct.float() / len(training_loader.dataset)

      # store epoch loss and accuracy
      running_loss_history.append(epoch_loss)
      running_correct_history.append(epoch_accuracy)

      # compute loss and accuracy for current epoch validation data
      val_epoch_loss = val_running_loss / len(validation_loader.dataset)
      val_epoch_accuracy = val_running_corrects.float() / len(validation_loader.dataset)

      # store epoch loss and accuracy for validation data
      val_running_loss_history.append(val_epoch_loss)
      val_running_corrects_history.append(val_epoch_accuracy)

      # print epoch results
      print('epoch: ', (i+1))
      print('training loss = {:.4f}, accuracy {:.4f}'.format(epoch_loss, epoch_accuracy.item()))
      print('validation loss = {:.4f}, validation accuracy {:.4f}'.format(val_epoch_loss, val_epoch_accuracy.item()))
  return (running_loss_history,running_correct_history,val_running_loss_history,val_running_corrects_history)

In [ ]:
# Calculate roc_auc scores for multiclass data
# From https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
  lb = preprocessing.LabelBinarizer()
  lb.fit(y_test)
  y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)

  return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
# function for calculating performance metric given model and test loader
def eval(model, test_loader):
    # initialize tensors to store output
    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()

    # set model to eval
    model.eval()
    for inputs, labels in test_loader:
        # pass to device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # calculate output and obtain predicted class
        y_prob = model(inputs)
        _, preds = torch.max(y_prob,1)

        # Pass to cpu and store data
        y_pred = torch.cat((y_pred,  preds.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, labels.detach().to('cpu')), dim=0)
    
    # calculate performance metrics
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    a = accuracy_score(y_true, y_pred)
    roc_auc = multiclass_roc_auc_score(y_true, y_pred)

    # store in dataframe to easily output
    result_df = pd.DataFrame({'Accuracy':[a],'Precision':[p],'Recall':[r],'F1 Score':[f], 'ROC AUC':[roc_auc]})
    return result_df

In [ ]:
# function for visualizing loss histories while training
def print_loss_histories(validation_loss_histories, tranformation, chart_label):
  for key in validation_loss_histories:
    plt.plot(validation_loss_histories[key], label=key)
    plt.legend()
    plt.title(chart_label + tranformation + " Validation Loss History")
  plt.show()

# function for visualizing accuracy histories
def print_accuracy_histories(validation_accuracy_histories, tranformation, chart_label):
  for key in validation_accuracy_histories:
    plt.plot(validation_accuracy_histories[key], label=key)
    plt.legend()
    plt.title(chart_label + tranformation + " Validation Accuracy History")
  plt.show()

In [ ]:
# function for returning pretrained models with new last layer
def get_models(classes):
  # ResNet
  resNetModel = models.resnet18(pretrained=True)
  # set grad to false so weights unchanged
  for param in resNetModel.parameters():
    param.requires_grad = False

  # replace last layer with new layer
  res_inputs = resNetModel.fc.in_features
  resNetModel.fc = nn.Linear(res_inputs, len(classes))
  # send to device
  resNetModel.to(device)

  # AlexNet
  alexNetModel = models.alexnet(pretrained=True)
  # set grad to false so weights unchanged
  for param in alexNetModel.features.parameters():
    param.requires_grad = False

  # replace last layer with new layer
  alex_in_features = alexNetModel.classifier[6].in_features 
  alexNetModel.classifier[6] = nn.Linear(in_features=alex_in_features, out_features=len(classes))
  # send to device
  alexNetModel.to(device)

  # DenseNet
  denseNetModel = models.densenet161(pretrained=True)
  # set grad to false so weights unchanged
  for param in denseNetModel.features.parameters():
    param.requires_grad = False

  # replace last layer with new layer
  dense_last_input = denseNetModel.classifier.in_features
  denseNetModel.classifier = nn.Linear(in_features=dense_last_input, out_features=len(classes))
  # send to device
  denseNetModel.to(device)

  # VGG
  vggModel = models.vgg19(pretrained=True)
  # set grad to false so weights unchanged
  for param in vggModel.features.parameters():
    param.requires_grad = False

  # replace last layer with new layer
  vgg_last_layer_input = vggModel.classifier[6].in_features
  vgg_new_layer = nn.Linear(in_features=vgg_last_layer_input, out_features=len(classes))
  vggModel.classifier[6] = vgg_new_layer
  # send to device
  vggModel.to(device)

  # return models with name labels
  return [
          { 
            'name': 'ResNet',
            'model': resNetModel
          }, 
          { 
            'name': 'AlexNet',
            'model':alexNetModel
          }, 
          {
            'name': 'DesnseNet',
            'model': denseNetModel
          }, 
          { 
            'name': 'VGG',
            'model': vggModel
          }
        ]

In [ ]:
# function to run training, and testing of all models using given transformation and list of labels
def run_experiment(transformation, classes):
  # initial variables to store running results
  validation_loss_histories = {}
  validation_accuracy_histories = {}
  eval_results = []

  epochs = 8
  lr = 0.0001

  # get pretrainied models
  models = get_models(classes)

  # get dataloaders based on transformation
  training_loader, validation_loader, test_loader = get_data_loaders(transformation)

  # View one image batch before training
  show_batch_images(training_loader)

  # loop through models list to train and store evaluation results
  for model in models:
    print('\n#################### Training ' + model['name'] + ' ####################')

    # train model and store results
    running_loss_history,running_correct_history,val_running_loss_history,val_running_corrects_history = train(model['model'], epochs, lr, training_loader, validation_loader)

    # add dictionary entry for model results
    validation_loss_histories[model['name']] = val_running_loss_history
    validation_accuracy_histories[model['name']] = val_running_corrects_history

    # store model test results with name
    eval_results.append({ 
        'name': model['name'],
        'results': eval(model['model'], test_loader)
    })

  # print results for each tested model
  for result in eval_results:
    print('\n#################### Test Results ' + result['name'] + ' ####################')
    print(result['results'])
    print('##########################################################\n')

  # determine chart label based on number of labels
  chart_label = 'Binary ' if len(classes) == 2 else 'Multi-class '

  # plot training loss and accuracy histories
  print_loss_histories(validation_loss_histories, transformation, chart_label)
  print_accuracy_histories(validation_accuracy_histories, transformation, chart_label)

  # Can retun models for reuse
  # return models

In [ ]:
# function for running all transformations using covid, normal and pneumonia data
def run_multi_class():
  # set flag for binary
  isBinary = False
  split_data(isBinary)

  # target labels
  classes = ('COVID', 'Normal', 'Pneumonia')

  # run experiment with all transformations
  run_experiment('Sobel', classes)
  run_experiment('Gaussian', classes)
  run_experiment('Random Perspective & Vertical Flip', classes)
  run_experiment('Unaugmented', classes)
  # clean up data to prevent interference with future tests
  clean_up()

# function for running all transformations using covid and normal data
def run_binary():
  # set flag for binary
  isBinary = True
  split_data(isBinary)

  # target labels
  classes = ('COVID', 'Normal')

  # run experiment with all transformations
  run_experiment('Sobel', classes)
  run_experiment('Gaussian', classes)
  run_experiment('Random Perspective & Vertical Flip', classes)
  run_experiment('Unaugmented', classes)
  # clean up data to prevent interference with future tests
  clean_up()

In [ ]:
# run expirement using covid, normal and pneumonia data
run_multi_class()

In [ ]:
# run expirement using covid and normal data
run_binary()